Step 1:- Import Required Libraries

In [ ]:
!pip uninstall -y huggingface_hub

In [2]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

Step 2:-Hugging face Authentication

In [ ]:
from huggingface_hub import login
login()

Step 3 :-Verify who

In [ ]:
from huggingface_hub import whoami
whoami()

Step 4:- Load mistral Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "mistralai/Mistral-7B-v0.1"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token  # FIX warning

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.float16,          # FIX deprecated warning
    device_map="auto"
)

print("🤖 Mistral Chatbot Ready! Type 'exit' to quit.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye 👋")
        break

    prompt = (
        "You are a helpful AI assistant.\n\n"
        f"Question: {user_input}\n"
        "Answer:"
    )

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True
    ).to(model.device)

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only model answer
    answer = response.split("Answer:")[-1].strip()

    print(f"Bot: {answer}\n")
